In [ ]:
pip install pandas NRCLex nltk textblob networkx 

### 1 Librerias

In [ ]:
import pandas as pd
from nrclex import NRCLex
import nltk
nltk.download('punkt')
print(nltk.data.path)

### 2 Carga de datos

In [2]:
# Cargar el archivo CSV
tweets_df = pd.read_csv('direccion/del/archivo.csv')

### 3 Analisis de emociones

In [4]:
# Función para analizar emociones en un tweet
def analyze_tweet_emotions(text):
    emotion = NRCLex(text)
    return emotion.raw_emotion_scores

In [ ]:
# Aplicar el análisis de emociones a la columna 'text'
tweets_df['emotions'] = tweets_df['text'].apply(analyze_tweet_emotions)

# Extraemos las emociones del diccionario a columnas y reemplazamos NaN por 0
tweets_df = pd.concat([tweets_df.drop(['emotions'], axis=1), tweets_df['emotions'].apply(pd.Series, dtype='float64')], axis=1).fillna(0)

tweets_df.head(5)


### 4 Visualizacion de datos

In [6]:
# path para guardar los gráficos
path = 'graficos/'

In [ ]:
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

# Select only numeric columns for correlation matrix
numeric_df = tweets_df.select_dtypes(include=[np.number])

# Crear una matriz de correlación
corr_matrix = numeric_df.corr()

# Crear una máscara para el triángulo superior
mask = np.triu(np.ones_like(corr_matrix, dtype=bool))

# Crear la visualización de la matriz de correlación
plt.figure(figsize=(10, 6))
sns.heatmap(corr_matrix, mask=mask, annot=True, cmap='coolwarm', vmin=-1, vmax=1, 
            square=True, linewidths=.5, cbar_kws={"shrink": .5})
plt.title('Triángulo superior de la matriz de correlación', fontsize=16)
plt.tight_layout()
plt.savefig(path + 'triangulo_matriz_correlacion.png')
plt.show()

In [ ]:
import networkx as nx
import matplotlib.pyplot as plt

def create_network(corr_matrix, threshold=0.5):
    G = nx.Graph()
    for i in range(len(corr_matrix.columns)):
        for j in range(i+1, len(corr_matrix.columns)):
            weight = corr_matrix.iloc[i, j]
            if abs(weight) >= threshold:
                G.add_edge(corr_matrix.columns[i], corr_matrix.columns[j], weight=weight)
    return G

G = create_network(corr_matrix)

plt.figure(figsize=(10, 6))
pos = nx.spring_layout(G)
edges = G.edges()
weights = [G[u][v]['weight'] for u,v in edges]

nx.draw(G, pos, with_labels=True, node_color='lightblue', 
        node_size=3000, font_size=10, font_weight='bold')

edge_labels = nx.get_edge_attributes(G, 'weight')
nx.draw_networkx_edge_labels(G, pos, edge_labels=edge_labels)

plt.title('Red de correlaciones entre emociones', fontsize=16)
plt.axis('off')
plt.tight_layout()
plt.savefig(path + 'red_correlaciones_emociones.png')
plt.show()

In [ ]:
# Creamos el gráfico de barras para ver que emociones encontramos en los tweets, normalizado
emotions = tweets_df.drop(['text'], axis=1).sum().sort_values(ascending=False)
emotions = emotions / emotions.sum()

# Creamos el gráfico de barras
plt.figure(figsize=(10, 6))
sns.barplot(x=emotions.values, y=emotions.index, palette='viridis')
plt.title('Distribución de Emociones en los Tweets', fontsize=16)
plt.xlabel('Proporción', fontsize=12)
plt.ylabel('Emoción', fontsize=12)
plt.tight_layout()
plt.savefig(path + 'distribucion_emociones_tweets.png')
plt.show()

### Guardamos la data

In [10]:
# Función para guardar el DataFrame a un nuevo CSV
def save_to_csv(df, save_path):
    df.to_csv(save_path, index=False)
    print(f"Archivo guardado en: {save_path}")

In [ ]:
# Guardar el DataFrame con las emociones
save_path = 'direccion/a/guardar/el/nuevo/archivo.csv'
save_to_csv(tweets_df, save_path)